In [9]:
import pandas as pd
import os
import openpyxl
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.styles import Border, Side, Alignment, Font, PatternFill

# === 1. Lê o arquivo TXT e converte em DataFrame ===
file_txt = "L:\\Publico\\Almoxarifado\\01 - Mapa_Producao\\mapa.TXT"

colunas =["Lote", "N Pedido", "Produto", "Cor Etiq", "QTD Vol", "Data Corte", "Data Emb", "Status", "N/A"]
df = pd.read_csv(file_txt, skiprows=23, delimiter=";", names=colunas)
df = df.dropna(subset=["N Pedido"])
df = df.drop(["QTD Vol", "Data Corte","Status", "N/A"], axis=1)
df ["Isopor"] = "-"
df ["Tecnoplac"] = "-"
df ["Lote_C2"] = "-"
df ["Vidro"] = "N/C"
ordem_colunas = ["N Pedido","Cor Etiq","Data Emb", "Lote", "Produto", "Isopor","Tecnoplac","Lote_C2","Vidro"]
df = df[ordem_colunas]
df.to_excel("C:\git\linea\Isopor\Outros\Mapa_Transformado.xlsx",index=False,engine='openpyxl')

# === 3. Merge com Mapa_Anterior.xlsx (se existir) ===
arquivo_anterior = "L:\\Publico\\Almoxarifado\\01 - Mapa_Producao\\Mapa_Producao.xlsx"
df_vidros = pd.read_excel("C:\git\linea\Isopor\Outros\Vidros.xlsx")

df_copia = pd.read_excel(arquivo_anterior)
df.to_excel("C:\git\linea\Isopor\Outros\Mapa_Copia.xlsx",index=False,engine='openpyxl')

if os.path.exists(arquivo_anterior):
    df_antigo = pd.read_excel(arquivo_anterior)

    # Faz o merge pelas colunas principais
    df = pd.merge(df, df_antigo[["Lote", "N Pedido", "Isopor", "Tecnoplac", "Lote_C2", "Vidro"]],
                  on=["Lote", "N Pedido"],
                  how="left",
                  suffixes=("", "_antigo"))
    
    # # Atualiza a coluna 'Vidro' apenas se o valor atual for "N/C"
    df['Vidro'] = df.apply(lambda row: verificar_vidro(row['Produto'], df_vidros['Lote'].values) if row['Vidro'] == "N/C" else row['Vidro'], axis=1)

    # # Normaliza os nomes para facilitar a comparação
    df['Produto'] = df['Produto'].str.upper()
    df_vidros['Lote'] = df_vidros['Lote'].str.upper()

    # Função para verificar se algum nome de lote de vidro está contido no nome do produto
    def verificar_vidro(produto, lotes_vidro):
        for lote in lotes_vidro:
            if lote in produto:
                return "Separar"
        return "N/C"

    # Aplica a função à coluna 'Produto'
    df['Vidro'] = df['Produto'].apply(lambda x: verificar_vidro(x, df_vidros['Lote'].values))

    for coluna in ["Isopor", "Tecnoplac", "Lote_C2", "Vidro"]:
        df[coluna] = df[f"{coluna}_antigo"].combine_first(df[coluna])
        df.drop(columns=[f"{coluna}_antigo"], inplace=True)


# === 4. Salva como Excel formatado ===
saida_excel = "L:\\Publico\\Almoxarifado\\01 - Mapa_Producao\\Mapa_Producao.xlsx"
df.to_excel(saida_excel, index=False, engine='openpyxl')

# === 5. Abre o arquivo e aplica formatações com openpyxl ===   
wb = openpyxl.load_workbook(saida_excel)
sheet = wb.active

sheet.title = "Visualização"

# Validação de dados (listas suspensas)
status_isopor = '"Mapeado, Parcial, Entregue, N/C, -"'
rule_isopor = DataValidation(type='list', formula1=status_isopor, allow_blank=True)
rule_isopor.error = "Entrada Inválida"
rule_isopor.errorTitle = "Selecione Corretamente"
rule_isopor.prompt = "Selecione a Lista"
rule_isopor.promptTitle = "Selecione a opção"
sheet.add_data_validation(rule_isopor)
rule_isopor.add("F2:F250")

status_tecnoplac = '"Entregue, Remessa, N/C, -"'
rule_tecnoplac = DataValidation(type='list', formula1=status_tecnoplac, allow_blank=True)
rule_tecnoplac.error = "Entrada Inválida"
rule_tecnoplac.errorTitle = "Selecione Corretamente"
rule_tecnoplac.prompt = "Selecione a Lista"
rule_tecnoplac.promptTitle = "Selecione a opção"
sheet.add_data_validation(rule_tecnoplac)
rule_tecnoplac.add("G2:G250")

status_lote = '"Parcial, Separado, Entregue, N/C, -"'
rule_lote = DataValidation(type='list', formula1=status_lote, allow_blank=True)
rule_lote.error = "Entrada Inválida"
rule_lote.errorTitle = "Selecione Corretamente"
rule_lote.prompt = "Selecione a Lista"
rule_lote.promptTitle = "Selecione a opção"
sheet.add_data_validation(rule_lote)
rule_lote.add("H2:H250")

status_vidro = '"Separar, Entregue, N/C, -"'
rule_vidro = DataValidation(type='list', formula1=status_vidro, allow_blank=True)
rule_vidro.error = "Entrada Inválida"
rule_vidro.errorTitle = "Selecione Corretamente"
rule_vidro.prompt = "Selecione a Lista"
rule_vidro.promptTitle = "Selecione a opção"
sheet.add_data_validation(rule_vidro)
rule_vidro.add("I2:I250")


# Bordas finas
borda = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
    for cell in row:
        cell.border = borda

# Tamanho das colunas
larguras = {'A': 12, 'B': 12, 'C': 13, 'D': 13, 'E': 60, 'F': 17, 'G': 17, 'H': 17,'I': 17}
for col, largura in larguras.items():
    sheet.column_dimensions[col].width = largura

# Centraliza células (exceto Produto)
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
    for col_idx in [0, 1, 2, 3, 4, 5, 6, 7, 8]: 
        row[col_idx].alignment = Alignment(horizontal='center')

# Fonte padrão
fonte_padrao = Font(size=10)
for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
    for cell in row:
        cell.font = fonte_padrao

# Cabeçalho formatado
fonte_cabecalho = Font(size=12, bold=True, color="FFFFFF")
fundo_roxo = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
for cell in sheet["A1:I1"][0]:
    cell.font = fonte_cabecalho
    cell.fill = fundo_roxo
    cell.alignment = Alignment(horizontal="center", vertical="center")

# Cores alternadas nas linhas
azul_claro = PatternFill(start_color="FBFDFE", end_color="FBFDFE", fill_type="solid")
azul_escuro = PatternFill(start_color="D9E5F2", end_color="D9E5F2", fill_type="solid")
for i, row in enumerate(sheet.iter_rows(min_row=2, max_row=sheet.max_row), start=0):
    fill = azul_claro if i % 2 == 0 else azul_escuro
    for cell in row:
            cell.fill = fill

sheet.freeze_panes = "A2"
wb.save(saida_excel)

# Salva uma cópia em outro local
caminho_copia = "C:\git\linea\Isopor\Mapa_Producao.xlsx"
wb.save(caminho_copia)

<>:20: SyntaxWarning: invalid escape sequence '\g'
<>:24: SyntaxWarning: invalid escape sequence '\g'
<>:27: SyntaxWarning: invalid escape sequence '\g'
<>:150: SyntaxWarning: invalid escape sequence '\g'
<>:20: SyntaxWarning: invalid escape sequence '\g'
<>:24: SyntaxWarning: invalid escape sequence '\g'
<>:27: SyntaxWarning: invalid escape sequence '\g'
<>:150: SyntaxWarning: invalid escape sequence '\g'
C:\Users\enzo.pedrica\AppData\Local\Temp\ipykernel_10764\1723045455.py:20: SyntaxWarning: invalid escape sequence '\g'
  df.to_excel("C:\git\linea\Isopor\Outros\Mapa_Transformado.xlsx",index=False,engine='openpyxl')
C:\Users\enzo.pedrica\AppData\Local\Temp\ipykernel_10764\1723045455.py:24: SyntaxWarning: invalid escape sequence '\g'
  df_vidros = pd.read_excel("C:\git\linea\Isopor\Outros\Vidros.xlsx")
C:\Users\enzo.pedrica\AppData\Local\Temp\ipykernel_10764\1723045455.py:27: SyntaxWarning: invalid escape sequence '\g'
  df.to_excel("C:\git\linea\Isopor\Outros\Mapa_Copia.xlsx",index=F